In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_files  
from sklearn.model_selection import train_test_split

from keras.preprocessing import image     
from tqdm import tqdm
from PIL import Image
from random import shuffle


import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Dense
from keras.applications.densenet import DenseNet121
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


import numpy as np
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
import os
import subprocess



In [ ]:
trainDf = pd.read_csv('CheXpert-v1.0-small/train.csv')

In [ ]:
# Remove anomalous dataline
trainDf = trainDf[trainDf.Sex != 'Unknown']

def pathToID(path):
    pathList = path.split('/')
    return pathList[2][7:]

def pathToStudy(path):
    pathList = path.split('/')
    return pathList[3][5:]

# Convert all labels to a series of one-hot encoded labels. 
# -1 is uncertain, 0 is negative, 1 is positive, nans are no mention of the disease in the text
trainDf = trainDf.fillna(0)
# N.B. this is replacing unknowns with true as per u-ones model here: https://arxiv.org/pdf/1901.07031.pdf
# This is essentialyl saying that if we're not sure of disease we say they have it. 
# Just to be on the safeside and have better recall as we care more about recall than precision
trainDf = trainDf.replace(-1,1) 


# Onehot encode the sex and the xray orientation
trainDf = trainDf.replace('Male',1)
trainDf = trainDf.replace('Female',0)
trainDf = trainDf.replace('Frontal',1)
trainDf = trainDf.replace('Lateral',0)

trainDf =trainDf.rename(index=str, columns={"Sex": "Male?",'Frontal/Lateral' :'Frontal1/Lateral0'})


#trainDf.insert(0,'Path', trainDf['Path'])
trainDf['Study'] = trainDf.Path.apply(pathToStudy)
trainDf['Patient ID'] = trainDf.Path.apply(pathToID)

# Rearrange Columns
cols = ['Patient ID', 'Study', 'Path', 'Age', 'Male?', 'Frontal1/Lateral0', 'AP/PA','No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']
trainDf = trainDf[cols]



# Preliminary Analysis

In [ ]:
# Shows age distribution of the data set. There are 3 0-olds and 7579 90 year olds. 
# Implies that over nineties were grouped together
ages = trainDf['Age'].value_counts()
plt.scatter(ages.keys(),ages.values)

In [ ]:
# How many people have no disease?
no_finding = trainDf['No Finding'].value_counts()
print(no_finding)

# Plot pie chart to show how much of the data is labelled with each character
values = no_finding.values
labels = ['Disease present','All Clear']

# Plot
plt.figure(figsize=(10, 5))
plt.title('Data Proportions', size=20)
plt.pie(values, labels=labels, # explode=explode,
        autopct='%1.1f%%', shadow=False, startangle=45)
 
plt.axis('equal')
plt.tight_layout()
plt.show()



In [ ]:
def load_dataset(path):
    # Load image files
    data = load_files(path)
    files = np.array(data['filenames'])
    return files


def path_to_tensor(img_path,inputSize):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, color_mode = "grayscale", target_size=inputSize)
    # convert PIL.Image.Image type to 3D tensor with shape (x, x, 1)
    x = image.img_to_array(img)
    data = np.asarray( img, dtype="int32" )
    # convert 2D tensor to 3D tensor with shape (1, X, x) and return 3D tensor
    return data.reshape(1,inputSize[0],inputSize[1])

def paths_to_tensor(img_paths, inputSize):
    # Convert paths to tensors for keras
    list_of_tensors = [path_to_tensor(img_path, inputSize) for img_path in img_paths]
    return np.array(list_of_tensors)


def path_to_tensor_channel_last_3colour(img_path,inputSize):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, color_mode = "grayscale", target_size=inputSize)
    # convert PIL.Image.Image type to 3D tensor with shape (x, x, 1)
    x = image.img_to_array(img)
    data = np.asarray( img, dtype="int32" )
    # convert 2D tensor to 3D tensor with shape (X, x, 3) and return 3D tensor
    # Repeat the same image 3 times to create 3 channels for densenet
    return np.stack((data,)*3, axis=-1)


def paths_to_tensor_channel_last_3colour(img_paths, inputSize):
    # Convert paths to tensors for keras
    list_of_tensors = [path_to_tensor_channel_last_3colour(img_path, inputSize) for img_path in img_paths]
    return np.array(list_of_tensors)

In [ ]:
inputSize = (224,224)

sample_size =20000 # 30k is the memory limit for the server

# Choose pathology to investigate
targetColumn = [15]
colName = trainDf.columns.tolist()[targetColumn[0]]
print(f"This model will be targetting {colName} column")


# Create balanced dataset with 50% pos examples and 50% neg examples, only take scans from the front
pos = trainDf[(trainDf[colName] == 1) & (trainDf['Frontal1/Lateral0'] == 1 ) & (trainDf['AP/PA'] == 'AP')]
neg = trainDf[(trainDf['No Finding'] == 1) & (trainDf['Frontal1/Lateral0'] == 1 ) & (trainDf['AP/PA'] == 'AP')]

posSample = pos.sample(int(sample_size/2))
negSample = neg.sample(int(sample_size/2))
sample = pd.concat([posSample,negSample])
x_train_paths, x_val_paths, y_train, y_val = train_test_split(sample.Path, sample[colName], stratify=sample[colName], random_state =2)

# The 3 channel option is required for the denseNet and other transfer learning models
# Single channel can be used on our none-transfer models

#x_train = paths_to_tensor(x_train_paths,inputSize)#.astype('float32')/255
x_train3Channel = paths_to_tensor_channel_last_3colour(x_train_paths,inputSize)#.astype('float32')/255

#y_train = trainDf.iloc[:training_no,targetColumn] # to do all labels: trainDf.iloc[:training_no,8:]
#x_val = paths_to_tensor(x_val_paths,inputSize)#.astype('float32')/255
x_val3Channel = paths_to_tensor_channel_last_3colour(x_val_paths,inputSize)#.astype('float32')/255

#y_val = trainDf.iloc[training_no:training_no+val_no,targetColumn]

# Deleting dataframes in order to save memory and avoid OOM errors. 
del trainDf
del posSample
del negSample
del x_train_paths
del x_val_paths


In [ ]:
# Trnsfer learning model
# put into separate cell as getting the dense net takes time 
# and we often only want to tweak the downstream architecutre 
denseNet = DenseNet121(input_shape=(224,224,3), include_top=True)
denseNet.layers.pop() # remov elast layer which has 1000 class softmax in it


In [ ]:
# Define the rest of the model
model2Layers = Flatten()(denseNet.layers[-2].output)
model2Layers = Dropout(0.3)(model2Layers)
model2Layers = Dense(1,activation='sigmoid')(model2Layers)
model2 = Model(input=denseNet.layers[0].input, output=model2Layers)
for i,layer in enumerate(model2.layers):
    # Don't train the first layers 
    if i < 428:
        #layer.trainable=False
        continue
    else:
        continue
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()
weightsFilePath2="weights2.best.hdf5"


In [ ]:
# Create checkpointer that saves the best model weights
checkpoint2 = ModelCheckpoint(weightsFilePath2, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
image_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15)

# Set model hyper params
batch_size = 16
epochs = 20

# Train model
history2 = model2.fit_generator(image_gen.flow(x_train3Channel, y_train, batch_size=batch_size),steps_per_epoch=len(x_train3Channel) / batch_size,  epochs = epochs, validation_data=(x_val3Channel, y_val), callbacks=[checkpoint2])
# Load best weights
model2.load_weights(weightsFilePath2)


In [ ]:
# Plot the history of this model
plt.plot(history2.history['acc'])
plt.plot(history2.history['val_acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'])
maxValAcc = max(history2.history['val_acc'])
trainAcc = history2.history['acc'][history2.history['val_acc'].index(maxValAcc)]
plt.savefig(f"Architecture10-{epochs}epochs-{colName}--trainAcc-{trainAcc}--valAcc-{maxValAcc}.png")
model2.save_weights(f"Architecture10-{epochs}epochs-{colName}--trainAcc-{trainAcc}--valAcc-{maxValAcc}.hdf5")
plt.show() 

## Results Analysis
In this section we analyse the results for each disease model. The models will load the weights from the best performing (as defined by validation accuracy) epoch. We then produce confusion matrices to calculate recall, precision and f1score. 

In [ ]:
# Load test data
testDf = pd.read_csv('CheXpert-v1.0-small/valid.csv')

# Remove anomalous dataline
testDf = testDf[testDf.Sex != 'Unknown']
# Drop this column as it has many more classifications than lit suggests and shouldn't matter greatly for a CNN

def pathToID(path):
    pathList = path.split('/')
    return pathList[2][7:]

def pathToStudy(path):
    pathList = path.split('/')
    return pathList[3][5:]

# Convert all labels to a series of one-hot encoded labels. 
# -1 is uncertain, 0 is negative, 1 is positive, nans are no mention of the disease in the text
testDf = testDf.fillna(0)
# N.B. this is replacing unknowns with true as per u-ones model here: https://arxiv.org/pdf/1901.07031.pdf
# This is essentialyl saying that if we're not sure of disease we say they have it. 
# Just to be on the safeside and have better recall as we care more about recall than precision
testDf = testDf.replace(-1,1) 


# Onehot encode the sex and the xray orientation
testDf = testDf.replace('Male',1)
testDf = testDf.replace('Female',0)
testDf = testDf.replace('Frontal',1)
testDf = testDf.replace('Lateral',0)

testDf =testDf.rename(index=str, columns={"Sex": "Male?",'Frontal/Lateral' :'Frontal1/Lateral0'})


#trainDf.insert(0,'Path', trainDf['Path'])
testDf['Study'] = testDf.Path.apply(pathToStudy)
testDf['Patient ID'] = testDf.Path.apply(pathToID)

# Rearrange Columns
cols = ['Patient ID', 'Study', 'Path', 'Age', 'Male?', 'Frontal1/Lateral0', 'AP/PA','No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices']
testDf = testDf[cols]

x_test3Channel = paths_to_tensor_channel_last_3colour(testDf.Path,inputSize)#.astype('float32')/255


In [ ]:
def analyseResults(model,x_test3Channel, testDf, disease, radiologistScores=None):
    # Analyses for a given pathology, getting roc curve and AUROC
    # Identify correct weights file to load
    file = [f for f in os.listdir('.') if os.path.isfile(f) and f"20epochs-{disease}"  in f and ".hdf5" in f]
    if len(file) != 1:
        print(f"Error: can't find single weights file, instead found: {file}")
    
    # Load in best weights
    model.load_weights(file[0])
    
    # Generate roc curve values
    model_predict_output = model.predict(x_test3Channel).flatten()
    y_test = testDf[disease]
    fpr, tpr, _ = roc_curve(y_test, model_predict_output)
    
    # Get AUROC
    auroc = roc_auc_score(y_test,model_predict_output)
    
    # Plot ROC
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr)
    
    # If radiologist scores as provided, plot them
    if radiologistScores != None:  
        plt.plot(radiologistScores[0][0],radiologistScores[0][1],'ro', label="Rad1") 
        plt.plot(radiologistScores[1][0],radiologistScores[1][1],'bo', label="Rad1") 
        plt.plot(radiologistScores[2][0],radiologistScores[2][1],'go', label="Rad1") 

    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.savefig(f"Architecture10-{disease}--AUROC-{auroc}.png")

    plt.show()
    return auroc

In [ ]:
diseases = ['Cardiomegaly','Atelectasis','Edema','Pneumonia','Pneumothorax']
# Radiologist scores from ChexPert paper. 
rads = [  [[0.05,0.48],[0.23,0.85],[0.11,0.70]], [[0.21,0.8],[0.18,0.71],[0.31,0.92]], [[0.09,0.63],[0.19,0.79],[0.07,0.58]] ]

# Plot for each pathology
for i,disease in enumerate(diseases):
    if i <= 2:
        auroc = analyseResults(model2,x_test3Channel,testDf,disease, radiologistScores=rads[i])
    else:
        auroc = analyseResults(model2,x_test3Channel,testDf,disease)
    print(f"{disease} AUROC: {auroc}")
    
    